In [1]:
%%configure -f
{
"conf": {"spark.pyspark.python": "python3", 
         "spark.pyspark.virtualenv.enabled": "true", 
         "spark.pyspark.virtualenv.type": "native", 
         "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"
        }, 
"kind": "pyspark",
"jars": [
        "s3://cse6242-project-data/spark-bootstrap/geo_wrapper_2.11-0.3.0.jar",
        "s3://cse6242-project-data/spark-bootstrap/geospark-1.3.0.jar",
        "s3://cse6242-project-data/spark-bootstrap/geospark-sql_2.3-1.3.0.jar"
    ]
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1586639853536_0004,pyspark,idle,Link,Link,


In [2]:
sc.install_pypi_package("geospark")
sc.list_packages()
print("Installed")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1586639853536_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached geospark-1.3.1-py3-none-any.whl (34.4 MB)
Processing /mnt/var/lib/livy/.cache/pip/wheels/84/30/e3/c51c5cd0229631e662d29d7b578a3e5949a4c8db033ffb70aa/pyspark-2.4.5-py2.py3-none-any.whl
  Using cached Shapely-1.7.0-cp36-cp36m-manylinux1_x86_64.whl (1.8 MB)
  Using cached findspark-1.3.0-py2.py3-none-any.whl (3.0 kB)
  Using cached attrs-19.3.0-py2.py3-none-any.whl (39 kB)
  Using cached py4j-0.10.7-py2.py3-none-any.whl (197 kB)

Package                    Version
-------------------------- -------
attrs                      19.3.0 
beautifulsoup4             4.8.1  
boto                       2.49.0 
findspark                  1.3.0  
geospark                   1.3.1  
jmespath                   0.9.4  
lxml                       4.4.2  
mysqlclient                1.4.6  
nltk                       3.4.5  
nose                       1.3.4  
numpy                      1.14.5 
pip                        20.0.2 
py-dateutil                2.2    
py4j                       0.

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Column as col, Row as row, SparkSession
from pyspark.sql.types import *

from geospark.register import upload_jars
from geospark.register import GeoSparkRegistrator

import shutil

# events_files = "/Users/istomins/tmp/data5/gzip/before2018/events/2020040101*"
events_files = "s3://cse6242-project-data/before2018/events/202002*"
# mentions_files = "/Users/istomins/tmp/data5/gzip/before2018/mentions/2020040101*"
mentions_files = "s3://cse6242-project-data/before2018/mentions/2020*"
# shape_file_location = "/Users/istomins/tmp/congressional-districts.csv"
shape_file_location = "s3://cse6242-project-data/congressional-districts.csv"

# party_file = "/Users/istomins/tmp/geoid-to-party.csv"
party_file = "s3://cse6242-project-data/geoid-to-party.csv"

# target_file = "/Users/istomins/tmp/data5/new_geo_df3.csv.gz"
target_file = "s3://cse6242-project-data/202002.csv"

upload_jars()

GeoSparkRegistrator.registerAll(spark)

sc = spark.sparkContext
# sqlContext = SQLContext(sc)
sqlContext = spark

export_schema = StructType([
    StructField("globaleventid", LongType(), True),
    StructField("day", IntegerType(), True),
    StructField("monthyear", IntegerType(), True),
    StructField("year", IntegerType(), True),
    StructField("fractiondate", DoubleType(), True),
    StructField("actor1code", StringType(), True),
    StructField("actor1name", StringType(), True),
    StructField("actor1countrycode", StringType(), True),
    StructField("actor1knowngroupcode", StringType(), True),
    StructField("actor1ethniccode", StringType(), True),
    StructField("actor1religion1code", StringType(), True),
    StructField("actor1religion2code", StringType(), True),
    StructField("actor1type1code", StringType(), True),
    StructField("actor1type2code", StringType(), True),
    StructField("actor1type3code", StringType(), True),
    StructField("actor2code", StringType(), True),
    StructField("actor2name", StringType(), True),
    StructField("actor2countrycode", StringType(), True),
    StructField("actor2knowngroupcode", StringType(), True),
    StructField("actor2ethniccode", StringType(), True),
    StructField("actor2religion1code", StringType(), True),
    StructField("actor2religion2code", StringType(), True),
    StructField("actor2type1code", StringType(), True),
    StructField("actor2type2code", StringType(), True),
    StructField("actor2type3code", StringType(), True),
    StructField("isrootevent", IntegerType(), True),
    StructField("eventcode", StringType(), True),
    StructField("eventbasecode", StringType(), True),
    StructField("eventrootcode", StringType(), True),
    StructField("quadclass", StringType(), True),
    StructField("goldsteinscale", DoubleType(), True),
    StructField("nummentions", IntegerType(), True),
    StructField("numsources", IntegerType(), True),
    StructField("numarticles", IntegerType(), True),
    StructField("avgtone", DoubleType(), True),
    StructField("actor1geo_type", IntegerType(), True),
    StructField("actor1geo_fullname", StringType(), True),
    StructField("actor1geo_countrycode", StringType(), True),
    StructField("actor1geo_adm1code", StringType(), True),
    StructField("actor1geo_adm2code", StringType(), True),
    StructField("actor1geo_lat", StringType(), True),
    StructField("actor1geo_long", StringType(), True),
    StructField("actor1geo_featureid", StringType(), True),
    StructField("actor2geo_type", IntegerType(), True),
    StructField("actor2geo_fullname", StringType(), True),
    StructField("actor2geo_countrycode", StringType(), True),
    StructField("actor2geo_adm1code", StringType(), True),
    StructField("actor2geo_adm2code", StringType(), True),
    StructField("actor2geo_lat", StringType(), True),
    StructField("actor2geo_long", StringType(), True),
    StructField("actor2geo_featureid", StringType(), True),
    StructField("actiongeo_type", IntegerType(), True),
    StructField("actiongeo_fullname", StringType(), True),
    StructField("actiongeo_countrycode", StringType(), True),
    StructField("actiongeo_adm1code", StringType(), True),
    StructField("actiongeo_adm2code", StringType(), True),
    StructField("actiongeo_lat", StringType(), True),
    StructField("actiongeo_long", StringType(), True),
    StructField("actiongeo_featureid", StringType(), True),
    StructField("dateadded", LongType(), True),
    StructField("sourceurl", StringType(), True),
])

mentions_schema = StructType([
    StructField("globaleventid", LongType(), True),
    StructField("eventtimedate", LongType(), True),
    StructField("mentiontimedate", LongType(), True),
    StructField("mentiontype", LongType(), True),
    StructField("mentionsourcename", StringType(), True),
    StructField("mentionidentifier", StringType(), True),
    StructField("sentenceid", LongType(), True),
    StructField("actor1charoffset", LongType(), True),
    StructField("actor2charoffset", LongType(), True),
    StructField("actioncharoffset", LongType(), True),
    StructField("inrawtext", StringType(), True),
    StructField("confidence", LongType(), True),
    StructField("mentiondoclen", LongType(), True),
    StructField("mentiondoctone", DoubleType(), True),
    StructField("mentiondoctranslationinfo", DoubleType(), True),
    StructField("extras", DoubleType(), True),
])

districts_schema = StructType([
    StructField("geometry", StringType(), True),
    StructField("STATEFP", StringType(), True),
    StructField("CD116FP", StringType(), True),
    StructField("AFFGEOID", StringType(), True),
    StructField("GEOID", StringType(), True),
    StructField("LSAD", StringType(), True),
    StructField("CDSESSN", StringType(), True),
    # StructField("ALAND", StringType(), True),
    # StructField("AWATER", StringType(), True),
])
party_schema = StructType([
    StructField("geoid", LongType(), False),
    StructField("party", StringType(), False),
])


# dir_files = spark_job_utils.get_s3_path_dir(type)


def parse(cols, schema):
    # cols = l.split("\t")
    for i, v in enumerate(cols):
        if not v or len(v.strip()) == 0:
            cols[i] = None
            continue
        t = type(schema.fields[i].dataType)
        if t in [IntegerType, LongType]:
            try:
                cols[i] = int(v)
            except Exception as e:
                print("Warning: %s cannot convert val %s to int. %s" % (schema.fieldNames()[i], v, e))
                cols[i] = None
        elif t in [DoubleType, FloatType]:
            try:
                cols[i] = float(v)
            except Exception as e:
                print("Warning: %s cannot convert val %s to float. %s" % (schema.fieldNames()[i], v, e))
                cols[i] = None
    return cols

def parse_events(l):
    return parse(l.split("\t"), export_schema)
def parse_mentions(l):
    return parse(l.split("\t"), mentions_schema)
def parse_party(l):
    return parse(l.split(","), party_schema)
def parse_districts(l):
    return parse(l.split("\t"), districts_schema)



# # congressional_rdd = GeoJsonReader.readToGeometryRDD(sc, shape_file_location)
# congressional_rdd = ShapefileReader.readToGeometryRDD(sc, shape_file_location)
# shutil.rmtree("/Users/istomins/tmp/geojson.json", ignore_errors=True)
# shutil.rmtree("/Users/istomins/tmp/geowkt", ignore_errors=True)
# shutil.rmtree("/Users/istomins/tmp/geocsv", ignore_errors=True)
# # congressional_rdd.saveAsGeoJSON("/Users/istomins/tmp/geojson.json")
# congressional_df = Adapter.toDf(congressional_rdd, spark)
congressional_rdd = sc.textFile(shape_file_location).map(parse_districts)
congressional_df = sqlContext.createDataFrame(congressional_rdd, districts_schema)
congressional_df.createOrReplaceTempView("districts")
congressional_df = spark.sql("""select ST_GeomFromWKT(d.geometry) as geometry,
     d.STATEFP,
     d.CD116FP,
     d.AFFGEOID,
     d.GEOID,
     d.LSAD,
     d.CDSESSN from districts d""")
congressional_df.createOrReplaceTempView("districts")
congressional_df.printSchema()
events_rdd = sc.textFile(events_files).map(parse_events)
events_df = sqlContext.createDataFrame(events_rdd, export_schema)
events_df.registerTempTable("events")
mentions_rdd = sc.textFile(mentions_files).map(parse_mentions)
mentions_df = sqlContext.createDataFrame(mentions_rdd, mentions_schema)
mentions_df.registerTempTable("mentions")

geoid_to_party_rdd = sc.textFile(party_file).map(parse_party)
geoid_to_party_df = sqlContext.createDataFrame(geoid_to_party_rdd, party_schema)
geoid_to_party_df.registerTempTable("geoid_party")

new_df = sqlContext.sql("""select e.globaleventid,
     first(e.day) day,
     first(e.eventcode) eventcode,
     first(e.eventbasecode) eventbasecode,
     first(e.eventrootcode) eventrootcode,
     first(e.actiongeo_fullname) actiongeo_fullname,
     first(e.actiongeo_countrycode) actiongeo_countrycode,
     first(e.actiongeo_lat) actiongeo_lat,
     first(e.actiongeo_long) actiongeo_long,
     ST_Point(CAST(first(e.actiongeo_long) AS Decimal(24,20)), CAST(first(e.actiongeo_lat) AS Decimal(24,20))) as loc_point,
     first(e.sourceurl) sourceurl,
     avg(m.mentiondoctone) as avgtone,
     count(m.globaleventid) as mentions_cnt
from events e
left join mentions m
on e.globaleventid = m.globaleventid
where e.actiongeo_lat is not null 
    and actiongeo_long is not null
    and actiongeo_countrycode = 'US'
group by e.globaleventid""")

new_df.registerTempTable("new_df")

new_df = sqlContext.sql("""select e.globaleventid,
     e.day,
     e.eventcode,
     e.eventbasecode,
     e.eventrootcode,
     e.actiongeo_fullname,
     e.actiongeo_countrycode,
     e.actiongeo_lat,
     e.actiongeo_long,
     e.sourceurl,
     e.avgtone,
     e.mentions_cnt,
     d.STATEFP,
     d.CD116FP,
     d.GEOID,
     d.LSAD,
     gp.party
from new_df e
join districts d
on ST_CONTAINS(d.geometry, e.loc_point)
left join geoid_party gp
on d.GEOID = gp.geoid""")


if not target_file.startswith("s3:"):
    shutil.rmtree(target_file, ignore_errors=True)
# new_df.write.parquet(target_dir, partitionBy=['monthyear'], compression="snappy")
new_df.write\
    .format("com.databricks.spark.csv") \
    .option("codec", "org.apache.hadoop.io.compress.GzipCodec") \
    .option("header", True) \
    .save(target_file)

print("Done")
# sc.stop()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…